In [1]:
import numpy as np

In [59]:
def polynom_matrix(a_min,a_max,b_min,b_max,seq_length): # USE THIS FOR COMPARISON TO FASTER FUNCTION
    a_range = a_max+1 - a_min
    b_range = b_max+1 - b_min
    
    Poly_matrix = np.fromfunction(lambda a,b,u,v: (a+a_min)*u*u + (b+b_min)*u*v, (a_range,b_range,seq_length,seq_length))
    
    return Poly_matrix

In [60]:
polynom_matrix(0,20,0,20,21)[1,1,2,3] # Should be 1*2*2 + 1*2*3 = 10

10.0

In [4]:
%timeit polynom_matrix(-10,10,-10,10,21)

4.65 ms ± 183 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%timeit make_P_array(-10,10,-10,10,21)

251 µs ± 2.45 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [48]:
polynom_matrix(0,20,0,20,21)[1,1,2,3]

10.0

In [61]:
make_P_array(0,20,0,20,21)[1,1,2,3]

10

Efficiently making a big matrix with entries $au^2 + buv$:

In [2]:
def make_P_array(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_squared_matrix = np.multiply.outer(a_range, u_range*u_range) # a x u
    au_tensor = np.broadcast_to(au_squared_matrix,(seq_length,b_len,a_len,seq_length)) # k x b x a x k
    #k,b,a,k --> a,b,k,k
    
    bu_matrix = np.multiply.outer(b_range,u_range) # b_len x seq_length
    buv_matrix = np.multiply.outer(bu_matrix,v_range) # b_len x seq_len x seq_len
    buv_tensor = np.broadcast_to(buv_matrix,(a_len,b_len,seq_length,seq_length))
    
    corrected_au_tensor = np.swapaxes(au_tensor, 0,2)
    #corrected_buv_tensor = np.swapaxes(buv_tensor, 2,3)
    
    tensor = corrected_au_tensor + buv_tensor
    
    P_array = np.swapaxes(tensor, 2,3)
    
    return P_array

In [16]:
%timeit make_P_array(-10,10,-10,10,21) # *racecar noises*

253 µs ± 2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [17]:
%timeit polynom_matrix(-10,10,-10,10,21) # *regular car noises*

4.82 ms ± 184 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [196]:
make_P_array(0,10,0,10,21)[1,1,1]

array([  0,   2,   6,  12,  20,  30,  42,  56,  72,  90, 110, 132, 156,
       182, 210, 240, 272, 306, 342, 380, 420])

In [217]:
polynom_matrix(0,10,0,10,21)[1,1,1]

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21.])

In [177]:
make_P_array(-5,5,-5,5,21)[3,8]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [  -2,    1,    4,    7,   10,   13,   16,   19,   22,   25,   28,
          31,   34,   37,   40,   43,   46,   49,   52,   55,   58],
       [  -8,   -2,    4,   10,   16,   22,   28,   34,   40,   46,   52,
          58,   64,   70,   76,   82,   88,   94,  100,  106,  112],
       [ -18,   -9,    0,    9,   18,   27,   36,   45,   54,   63,   72,
          81,   90,   99,  108,  117,  126,  135,  144,  153,  162],
       [ -32,  -20,   -8,    4,   16,   28,   40,   52,   64,   76,   88,
         100,  112,  124,  136,  148,  160,  172,  184,  196,  208],
       [ -50,  -35,  -20,   -5,   10,   25,   40,   55,   70,   85,  100,
         115,  130,  145,  160,  175,  190,  205,  220,  235,  250],
       [ -72,  -54,  -36,  -18,    0,   18,   36,   54,   72,   90,  108,
         126,  144,  162,  180,  198,  216,  234,  252,  270,  288],

We want to work on getting a matrix of sequences of $\tau(n)$ where
$$
\tau(n) = \frac{1}{n^2(n-1)}\sum_{u+v=n}(au^2 + buv)\tau(u)\tau(v).
$$
For the original sequence of $\sigma_n$, we'd use $a = -24$ and $b = 36$.

In [11]:
def make_sequences(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = make_P_array(a_min,a_max,b_min,b_max,seq_length) # Big array for the polynomial values
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len, b_len, seq_length)).copy() # Big array for the sequences
    
    for a in range(a_len):
        for b in range(b_len):
            flipped_array = np.flipud(coeff_array[a,b])
            for n in range(2,seq_length):
                diag = np.diagonal(flipped_array, -seq_length+n+1)[1:n]
                vec_A = diag*seq_array[a,b,1:n]
                vec_B = seq_array[a,b,n-1:0:-1]
                seq_array[a,b,n] = (1/((n*n)*(n-1)))*np.dot(vec_A,vec_B)
                
    return seq_array

In [8]:
%timeit make_sequences(-10,10,-10,10,20) # Not great, but I got it

42.9 ms ± 502 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
make_sequences(-24,-24,36,36,20) # Nice

array([[[ 0.,  1.,  3.,  4.,  7.,  6., 12.,  8., 15., 13., 18., 12.,
         28., 14., 24., 24., 31., 18., 39., 20.]]])

## From Here:

Finally get to start playing with some higher order polynomials. Thankfully, their respective matrices are still 4d and can be constructed in a similar manner, and the process for getting the sequences won't be much slower than it already is. So that's not nothing.

#### Linear

$$
\tau(n) = \frac{1}{n(n-1)}\sum_{u+v=n}(au+bv)\tau(u)\tau(v)
$$

In [3]:
def deg_1_array(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_array = np.multiply.outer(a_range,u_range) # a x u
    bv_array = np.multiply.outer(b_range,v_range) # b x v
    
    temp_array = np.add.outer(au_array, bv_array) # a x u x b x v
    
    final_array = np.swapaxes(temp_array, 1,2)
    
    return final_array



def deg_1_sqnc(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = deg_1_array(a_min,a_max,b_min,b_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,seq_length)).copy()
    
    for a in range(a_len):
        for b in range(b_len):
            flipped_array = np.flipud(coeff_array[a,b])
            for n in range(2,seq_length):
                diag = np.diagonal(flipped_array, -seq_length+n+1)[1:n]
                vec_A = diag*seq_array[a,b,1:n]
                vec_B = seq_array[a,b,n-1:0:-1]
                seq_array[a,b,n] = (1/(n*(n-1)))*np.dot(vec_A,vec_B)
                
    return seq_array

In [4]:
X = deg_1_sqnc(0,40,0,40,21)

In [86]:
X[19,3]

array([0.00000000e+00, 1.00000000e+00, 1.10000000e+01, 1.21000000e+02,
       1.33100000e+03, 1.46410000e+04, 1.61051000e+05, 1.77156100e+06,
       1.94871710e+07, 2.14358881e+08, 2.35794769e+09, 2.59374246e+10,
       2.85311671e+11, 3.13842838e+12, 3.45227121e+13, 3.79749834e+14,
       4.17724817e+15, 4.59497299e+16, 5.05447028e+17, 5.55991731e+18,
       6.11590904e+19])

In [89]:
X[0,2] # a = 0, b = 2

array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1.])

In [91]:
X[0,4] #a = 0, b = 4

array([0.00000e+00, 1.00000e+00, 2.00000e+00, 4.00000e+00, 8.00000e+00,
       1.60000e+01, 3.20000e+01, 6.40000e+01, 1.28000e+02, 2.56000e+02,
       5.12000e+02, 1.02400e+03, 2.04800e+03, 4.09600e+03, 8.19200e+03,
       1.63840e+04, 3.27680e+04, 6.55360e+04, 1.31072e+05, 2.62144e+05,
       5.24288e+05])

In [93]:
X[0,6]

array([0.00000000e+00, 1.00000000e+00, 3.00000000e+00, 9.00000000e+00,
       2.70000000e+01, 8.10000000e+01, 2.43000000e+02, 7.29000000e+02,
       2.18700000e+03, 6.56100000e+03, 1.96830000e+04, 5.90490000e+04,
       1.77147000e+05, 5.31441000e+05, 1.59432300e+06, 4.78296900e+06,
       1.43489070e+07, 4.30467210e+07, 1.29140163e+08, 3.87420489e+08,
       1.16226147e+09])

Looks likeif $a = 0$ and $b = 2k$ for $k \in \mathbb{Z}$, we get a geometric sequence.

In [95]:
X[1,3] # a = 1, b = 3, a+b = 4 ---> 2^n

array([0.00000e+00, 1.00000e+00, 2.00000e+00, 4.00000e+00, 8.00000e+00,
       1.60000e+01, 3.20000e+01, 6.40000e+01, 1.28000e+02, 2.56000e+02,
       5.12000e+02, 1.02400e+03, 2.04800e+03, 4.09600e+03, 8.19200e+03,
       1.63840e+04, 3.27680e+04, 6.55360e+04, 1.31072e+05, 2.62144e+05,
       5.24288e+05])

In [96]:
X[1,5] # a = 1, b = 5, a+b = 6 ---> 3^n

array([0.00000000e+00, 1.00000000e+00, 3.00000000e+00, 9.00000000e+00,
       2.70000000e+01, 8.10000000e+01, 2.43000000e+02, 7.29000000e+02,
       2.18700000e+03, 6.56100000e+03, 1.96830000e+04, 5.90490000e+04,
       1.77147000e+05, 5.31441000e+05, 1.59432300e+06, 4.78296900e+06,
       1.43489070e+07, 4.30467210e+07, 1.29140163e+08, 3.87420489e+08,
       1.16226147e+09])

In [97]:
X[1,7] # a = 1, b = 7, a+b = 8 ---> 4^n

array([0.00000000e+00, 1.00000000e+00, 4.00000000e+00, 1.60000000e+01,
       6.40000000e+01, 2.56000000e+02, 1.02400000e+03, 4.09600000e+03,
       1.63840000e+04, 6.55360000e+04, 2.62144000e+05, 1.04857600e+06,
       4.19430400e+06, 1.67772160e+07, 6.71088640e+07, 2.68435456e+08,
       1.07374182e+09, 4.29496730e+09, 1.71798692e+10, 6.87194767e+10,
       2.74877907e+11])

In [106]:
X[2,2] # a = 2, b = 2, a+b = 4 ---> 2^n

array([0.00000e+00, 1.00000e+00, 2.00000e+00, 4.00000e+00, 8.00000e+00,
       1.60000e+01, 3.20000e+01, 6.40000e+01, 1.28000e+02, 2.56000e+02,
       5.12000e+02, 1.02400e+03, 2.04800e+03, 4.09600e+03, 8.19200e+03,
       1.63840e+04, 3.27680e+04, 6.55360e+04, 1.31072e+05, 2.62144e+05,
       5.24288e+05])

In [6]:
X[3,7] # a = 3, b = 7, a+b = 10 ---> 5^n

array([0.00000000e+00, 1.00000000e+00, 5.00000000e+00, 2.50000000e+01,
       1.25000000e+02, 6.25000000e+02, 3.12500000e+03, 1.56250000e+04,
       7.81250000e+04, 3.90625000e+05, 1.95312500e+06, 9.76562500e+06,
       4.88281250e+07, 2.44140625e+08, 1.22070312e+09, 6.10351563e+09,
       3.05175781e+10, 1.52587891e+11, 7.62939453e+11, 3.81469727e+12,
       1.90734863e+13])

In [7]:
X[19,17] # a = 19, b = 17, a+b = 36 ---> 18^n

array([0.00000000e+00, 1.00000000e+00, 1.80000000e+01, 3.24000000e+02,
       5.83200000e+03, 1.04976000e+05, 1.88956800e+06, 3.40122240e+07,
       6.12220032e+08, 1.10199606e+10, 1.98359290e+11, 3.57046723e+12,
       6.42684101e+13, 1.15683138e+15, 2.08229649e+16, 3.74813368e+17,
       6.74664062e+18, 1.21439531e+20, 2.18591156e+21, 3.93464081e+22,
       7.08235345e+23])

It sort of looks like if $a+b=k$ is even, then $\tau(n)$ is a geometric sequence of the form $\left(\frac{k}{2}\right)^n$...

#### Quadratic

$$
\tau(n) = \frac{1}{n^2(n-1)}\sum_{u+v=n}(au^2 + buv)\tau(u)\tau(v)
$$

In [108]:
def deg_2_array(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_squared_matrix = np.multiply.outer(a_range, u_range*u_range) # a x u
    au_tensor = np.broadcast_to(au_squared_matrix,(seq_length,b_len,a_len,seq_length)) # k x b x a x k
    #k,b,a,k --> a,b,k,k
    
    bu_matrix = np.multiply.outer(b_range,u_range) # b_len x seq_length
    buv_matrix = np.multiply.outer(bu_matrix,v_range) # b_len x seq_len x seq_len
    buv_tensor = np.broadcast_to(buv_matrix,(a_len,b_len,seq_length,seq_length))
    
    corrected_au_tensor = np.swapaxes(au_tensor, 0,2)
    #corrected_buv_tensor = np.swapaxes(buv_tensor, 2,3)
    
    tensor = corrected_au_tensor + buv_tensor
    
    P_array = np.swapaxes(tensor, 2,3)
    
    return P_array

def deg_2_sqnc(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = deg_2_array(a_min,a_max,b_min,b_max,seq_length) # Big array for the polynomial values
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len, b_len, seq_length)).copy() # Big array for the sequences
    
    for a in range(a_len):
        for b in range(b_len):
            flipped_array = np.flipud(coeff_array[a,b])
            for n in range(2,seq_length):
                diag = np.diagonal(flipped_array, -seq_length+n+1)[1:n]
                vec_A = diag*seq_array[a,b,1:n]
                vec_B = seq_array[a,b,n-1:0:-1]
                seq_array[a,b,n] = (1/((n*n)*(n-1)))*np.dot(vec_A,vec_B)
                
    return seq_array

In [112]:
Z = deg_2_sqnc(0,40,0,40,21)

In [117]:
Z[2,2] # a = 2, b = 2 ---> 1^n :P

array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1.])

In [121]:
Z[4,4] # a = 4, b = 4 ---> 2^n

array([0.00000e+00, 1.00000e+00, 2.00000e+00, 4.00000e+00, 8.00000e+00,
       1.60000e+01, 3.20000e+01, 6.40000e+01, 1.28000e+02, 2.56000e+02,
       5.12000e+02, 1.02400e+03, 2.04800e+03, 4.09600e+03, 8.19200e+03,
       1.63840e+04, 3.27680e+04, 6.55360e+04, 1.31072e+05, 2.62144e+05,
       5.24288e+05])

In [144]:
Z[6,18] # Almost...

array([0.00000000e+00, 1.00000000e+00, 6.00000000e+00, 3.40000000e+01,
       1.91000000e+02, 1.06962000e+03, 5.98142400e+03, 3.34232456e+04,
       1.86679787e+05, 1.04237015e+06, 5.81920239e+06, 3.24822963e+07,
       1.81295813e+08, 1.01180623e+09, 5.64654620e+09, 3.15100935e+10,
       1.75833510e+11, 9.81164092e+11, 5.47484582e+12, 3.05487955e+13,
       1.70454934e+14])

In [150]:
Z[14,14] ---> 7^n

array([0.00000000e+00, 1.00000000e+00, 7.00000000e+00, 4.90000000e+01,
       3.43000000e+02, 2.40100000e+03, 1.68070000e+04, 1.17649000e+05,
       8.23543000e+05, 5.76480100e+06, 4.03536070e+07, 2.82475249e+08,
       1.97732674e+09, 1.38412872e+10, 9.68890104e+10, 6.78223073e+11,
       4.74756151e+12, 3.32329306e+13, 2.32630514e+14, 1.62841360e+15,
       1.13988952e+16])

So far, all I'm seeing is that if $a=b$ and $a$ and $b$ are even, then we get a geometric sequence, this time of the form $\left(\frac{a}{2}\right)^n$.

In [154]:
Y = deg_2_sqnc(-30,10,0,40,21)

In [185]:
Y[34]

array([[0.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.11111111e+00,
        1.25925926e+00, 1.43407407e+00, 1.63516049e+00, 1.86487769e+00,
        2.12676945e+00, 2.42516833e+00, 2.76512033e+00, 3.15241721e+00,
        3.59367463e+00, 4.09643689e+00, 4.66930236e+00, 5.32206902e+00,
        6.06590148e+00, 6.91352162e+00, 7.87942615e+00, 8.98013449e+00,
        1.02344713e+01],
       [0.00000000e+00, 1.00000000e+00, 1.25000000e+00, 1.66666667e+00,
        2.24826389e+00, 3.04201389e+00, 4.11989583e+00, 5.58152006e+00,
        7.56254558e+00, 1.02470495e+01, 1.38845562e+01, 1.88131996e+01,
        2.54911394e+01, 3.45391403e+01, 4.67982681e+01, 6.34080619e+01,
        8.59124495e+01, 1.16403248e+02, 1.57714589e+02, 2.13686335e+02,
        2.89520974e+02],
       [0.00000000e+00, 1.00000000e+00, 1.50000000e+00, 2.33333333e+00,
        3.65277778e+00, 5.72833333e+00, 8.98877778e+00, 1.41085908e+01,
        2.21471486e+01, 3.47678371e+01, 5.45822207e+01, 8.56904339e+01,
        1.3452

#### Cubic

$$
\tau(n) = \frac{1}{n^3(n-1)}\sum_{u+v=n}(au^3 + bu^2v)\tau(u)\tau(v)
$$

In [9]:
def deg_3_array(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_squared_matrix = np.multiply.outer(a_range, u_range*u_range*u_range) # a x u^3
    au_tensor = np.broadcast_to(au_squared_matrix,(seq_length,b_len,a_len,seq_length)) # k x b x a x k
    #k,b,a,k --> a,b,k,k
    
    bu_matrix = np.multiply.outer(b_range,u_range*u_range) # b x u^2
    buv_matrix = np.multiply.outer(bu_matrix,v_range) # b x u^2 x v
    buv_tensor = np.broadcast_to(buv_matrix,(a_len,b_len,seq_length,seq_length)) # a x b x k x k
    
    corrected_au_tensor = np.swapaxes(au_tensor, 0,2)
    #corrected_buv_tensor = np.swapaxes(buv_tensor, 2,3)
    
    tensor = corrected_au_tensor + buv_tensor
    
    P_array = np.swapaxes(tensor, 2,3)
    
    return P_array

def deg_3_sqnc(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = deg_3_array(a_min,a_max,b_min,b_max,seq_length) # Big array for the polynomial values
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len, b_len, seq_length)).copy() # Big array for the sequences
    
    for a in range(a_len):
        for b in range(b_len):
            flipped_array = np.flipud(coeff_array[a,b])
            for n in range(2,seq_length):
                diag = np.diagonal(flipped_array, -seq_length+n+1)[1:n]
                vec_A = diag*seq_array[a,b,1:n]
                vec_B = seq_array[a,b,n-1:0:-1]
                seq_array[a,b,n] = (1/((n*n*n)*(n-1)))*np.dot(vec_A,vec_B)
                
    return seq_array

In [11]:
A = deg_3_sqnc(-10,10,-10,10,21)

In [34]:
A[8,4] # a = -2, b = -6?

array([ 0.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
        1., -1.,  1., -1.,  1., -1.,  1., -1.])

In [39]:
A[12,16]

array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1.])

In [10]:
B = deg_3_sqnc(0,40,0,40,21)

In [73]:
B[4,12] # a = 4 ---> 2^n

array([0.00000e+00, 1.00000e+00, 2.00000e+00, 4.00000e+00, 8.00000e+00,
       1.60000e+01, 3.20000e+01, 6.40000e+01, 1.28000e+02, 2.56000e+02,
       5.12000e+02, 1.02400e+03, 2.04800e+03, 4.09600e+03, 8.19200e+03,
       1.63840e+04, 3.27680e+04, 6.55360e+04, 1.31072e+05, 2.62144e+05,
       5.24288e+05])

In [81]:
B[10,30] # a = 10 ---> 5^n

array([0.00000000e+00, 1.00000000e+00, 5.00000000e+00, 2.50000000e+01,
       1.25000000e+02, 6.25000000e+02, 3.12500000e+03, 1.56250000e+04,
       7.81250000e+04, 3.90625000e+05, 1.95312500e+06, 9.76562500e+06,
       4.88281250e+07, 2.44140625e+08, 1.22070312e+09, 6.10351562e+09,
       3.05175781e+10, 1.52587891e+11, 7.62939453e+11, 3.81469727e+12,
       1.90734863e+13])

In [11]:
B[6,18] # a = 6 ---> 3^n

array([0.00000000e+00, 1.00000000e+00, 3.00000000e+00, 9.00000000e+00,
       2.70000000e+01, 8.10000000e+01, 2.43000000e+02, 7.29000000e+02,
       2.18700000e+03, 6.56100000e+03, 1.96830000e+04, 5.90490000e+04,
       1.77147000e+05, 5.31441000e+05, 1.59432300e+06, 4.78296900e+06,
       1.43489070e+07, 4.30467210e+07, 1.29140163e+08, 3.87420489e+08,
       1.16226147e+09])

$$
\frac{1}{n^3(n-1)}\sum_{u+v=n}(6u^3 + 18u^2v)\tau(u)\tau(v)
$$

For $\deg(P(u,v)) = 3$, it looks like we get a geometric sequence if $a = 2k, \ k \in \mathbb{Z}$, and $b = 3a$. It looks like the geometric sequence is again of the form  $\left(\frac{a}{2}\right)^n$. I'm probably missing something that makes this obvious, but that's kind of fun.